Todo

- Exportovat seznam seznamových sloupců, aby ho další sešit v pipelině načítal automaticky.

- Ještě mnoho sloupců seznamových, kvůli čárkám ", sro" nutno upravit při scrapování.

- Ukazují se exulantské filmy! Zároveň tam ale chceme nechat koprodukce. (Ten samý problém u herectva.)

- Premiéra do datetime.

- Rok výroby do dvou sloupců.

In [1]:
import pandas as pd

In [2]:
df = pd.read_json("filmy_raw.json")

In [3]:
df.columns.to_list()

['Film',
 'Země původu',
 'Copyright',
 'Rok výroby',
 'Premiéra',
 'Minutáž',
 'Režie',
 'Kategorie',
 'Žánr',
 'Typologie',
 'Originální název',
 'Český název',
 'Anglický název',
 'Souběžný název',
 'Pracovní název',
 'Pomocná režie',
 'Asistent režie',
 'Původní filmový námět',
 'Scénář',
 'Dramaturg',
 'Kamera',
 'Asistent kamery',
 'Architekt',
 'Návrhy kostýmů',
 'Masky',
 'Střih',
 'Zvuk',
 'Producent',
 'Vedoucí výroby',
 'Zástupce vedoucího výroby',
 'Odborný poradce',
 'Lokace',
 'Distribuční slogan',
 'Výrobce',
 'S podporou',
 'Nositelé copyrightu',
 'Distribuce',
 'Délka',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Anotace',
 'Obsah',
 'Technický scénář',
 'Výtvarník',
 'Animace',
 'Triky',
 'Postprodukce',
 'Návrhy titulků',
 'Pedagogické vedení',
 'Studentský film',
 'Původní metráž',
 'Mezititulky',
 'Vizuální efekty',
 'Vedoucí produkce',
 'Předloha',
 'Kaskadéři',
 'pás

In [4]:
def citlivy_orez(x):
    if isinstance(x, str):
        x = x.strip()
    if isinstance(x, list):
        try:
            x = [y.strip() for y in x]
        except:
            pass
    return(x)

In [5]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [6]:
df = df.apply(lambda x: citlivy_orez(x))

In [7]:
seznamove_sloupce = ["Žánr",
                     "Režie",
                     "Scénář",
                     "Mluveno",
                     "Barva",
                     "Zvukový systém/formát",
                     "Země původu",
                     "Úvodní/závěrečné titulky",
                     "Distribuční nosič",
                     "Země původu",
                     "Typologie"]

In [8]:
for s in seznamove_sloupce:
    df[s] = df[s].apply(lambda x: do_seznamu(x))

In [9]:
df["Minutáž"] = df["Minutáž"].apply(lambda x: str(x).replace(" min",""))
df["Minutáž"] = pd.to_numeric(df["Minutáž"], errors="coerce")

In [10]:
def seznam_lokaci(lokace_hrube):
    lokace_ciste = []
    if lokace_hrube:
        lokace_hrube = lokace_hrube.split("),")
        for x in lokace_hrube:
            if "(" in x:
                x = x + ")"
            x = x.strip()
            x = x.replace("))",")")
            if x[0:2] == ", ":
                x = x[2:]
            if x != ")":
                lokace_ciste.append(x)
    return(lokace_ciste)

In [11]:
df["Lokace"] = df["Lokace"].apply(lambda x: seznam_lokaci(x))

In [12]:
df = df.sort_values(by = "Copyright")

In [13]:
df.to_json("filmy.json", orient="records")

## Další experimenty

In [14]:
pd.set_option('display.max_colwidth', None)

In [15]:
df["Premiéra"].sample(60)

677                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [16]:
import re

In [17]:
def premiera(x, jaka):
    try:
        if f"{jaka}premiéra" in x:
            x = x.split(f"{jaka}premiéra")[1]
            try:
                datum = re.search('.*?\d{4}', x).group().replace(f"{jaka}premiéra","").strip()
                # datum = pd.to_datetime(datum, errors="coerce", format = "%d. %m. %Y")
            except:
                datum = ""
            try:
                misto = re.search('\((.*?)\)', x).group().replace("(","").replace(")","").strip()
            except AttributeError:
                misto = ""
        else:
            datum = ""
            misto = ""
    except TypeError:
        datum = ""
        misto = ""
    return [datum, misto]

In [18]:
df["Slavnostní premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "slavnostní ")[0])
df["Slavnostní premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "slavnostní ")[1])
df["Distribuční premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "distribuční ")[0])
df["Distribuční premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "distribuční ")[1])
df["Festivalová premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "festivalová ")[0])
df["Festivalová premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "festivalová ")[1])
df["Zahraniční premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "zahraniční ")[0])
df["Zahraniční premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "zahraniční ")[1])
df["Premiéra bez upřesnění (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "")[0])

In [19]:
df["Premiéra bez upřesnění (datum)"].nunique()

2957

In [20]:
df.sort_values(by='Premiéra bez upřesnění (datum)',ascending=False)

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,cyklus,Slavnostní premiéra (datum),Slavnostní premiéra (místo),Distribuční premiéra (datum),Distribuční premiéra (místo),Festivalová premiéra (datum),Festivalová premiéra (místo),Zahraniční premiéra (datum),Zahraniční premiéra (místo),Premiéra bez upřesnění (datum)
1945,Lásky jedné plavovlásky (396651),Československo,1965,1965,"festivalová premiéra srpen 1965 (26. mezinárodní filmový festival Benátky, Itálie) premiéra 12. 11. 1965 /nepřístupný mládeži/ (celostátní) předpremiéra 12. 11. 1965 (kino Sevastopol /2 týdny/, Praha) premiéra 26. 11. 1965 /nepřístupný mládeži/ (kina Jalta /8 týdnů/, Kotva /3 týdny/, Odboj /1 týden od 17. 12./, Flora /1 týden od 25. 12./, Oko /1 týden od 31. 12./ a Revoluce /1 týden od 7. 1. 1966/, Praha) obnovená premiéra 5. 6. 2019 /přístupné bez omezení/ (kino Ponrepo, Praha)",77.0,Miloš Forman (7872),film,"[komedie, hořká komedie]","[hraný, distribuční]",...,None,,,,,srpen 1965,"26. mezinárodní filmový festival Benátky, Itálie",,,srpen 1965
510,Utrpením ke slávě (395208),Československo,1919,1919,premiéra léto 1919 /nepřístupný mládeži/,NaN,Richard F. Branald (10106),film,historický,"[hraný, distribuční]",...,None,,,,,,,,,léto 1919
3739,Kočárová doprava (401687),Rakousko-Uhersko,1908,1908,premiéra léto 1908 (Jubilejní výstava obchodní a živnostenské komory na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,léto 1908
3748,První sjezd českých střeleckých jednot v Praze (401696),Rakousko-Uhersko,1908,1908,premiéra léto 1908 (Jubilejní výstava obchodní a živnostenské komory na pražském Výstavišti),NaN,Jan Kříženecký (127419),film,"[reportáž, sportovní]","[dokumentární, distribuční]",...,None,,,,,,,,,léto 1908
3745,Průvod delegací v tradičních krojích (401693),Rakousko-Uhersko,1908,1908,premiéra léto 1908 (Jubilejní výstava obchodní a živnostenské komory na pražském Výstavišti),1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,léto 1908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,Mladá láska (396196),Československo,1954,1954,None,8.0,Jan Valášek (20326),film,příběh,"[hraný, nedistribuční, studentský]",...,None,,,,,,,,,
3107,Tajemství úspěchu (398554),Československo,1934,1934,premiéra neproběhla,2.0,Arnošt Továrek (127532),film,reklamní,"[animovaný, distribuční]",...,None,,,,,,,,,
1534,Život (396238),Československo,1956,1956,None,12.0,Ludvík Ráža (5281),film,etuda,"[hraný, nedistribuční, studentský]",...,None,,,,,,,,,
3418,Dinosauři (400541),Česká republika,2013,2013,None,6.0,Dao Thi Thuy Linh (124417),film,komedie,"[animovaný, distribuční, krátkometrážní]",...,None,,,,,,,,,


In [21]:
df[df["Copyright"] == 1898].sort_values(by="Premiéra bez upřesnění (datum)")

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,cyklus,Slavnostní premiéra (datum),Slavnostní premiéra (místo),Distribuční premiéra (datum),Distribuční premiéra (místo),Festivalová premiéra (datum),Festivalová premiéra (místo),Zahraniční premiéra (datum),Zahraniční premiéra (místo),Premiéra bez upřesnění (datum)
3767,Útok pražského dělostřelectva (401716),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inženýrství na pražském Výstavišti),1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
412,Smích a pláč (395110),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,anekdota,"[hraný, distribuční]",...,None,,,,,,,,,07/1898
3764,Rychlovlak v Podbabí (401713),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inženýrství na pražském Výstavišti),1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
3762,Přenesení kolébky Františka Palackého z Hodslavic na Výstaviště (401710),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
3766,Svatojanská pouť v českoslovanské vesnici (401715),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3763,Purkyňovo náměstí na Královských Vinohradech (401712),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a inženýrství na pražském Výstavišti),1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3761,Polední výstřel na Mariánských hradbách (401709),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3758,Defilování vojska o Božím těle na Královských Hradčanech (401706),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
413,Výstavní párkař a lepič plakátů (395111),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,fraška,"[hraný, distribuční]",...,None,,,,,,,,,24. 7. 1898
3768,Voltýžování jízdního odboru Sokola pražského (401717),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a inženýrství na pražském Výstavišti) obnovená premiéra 30. 8. 2018 /přístupné bez omezení/,1.0,Jan Kříženecký (127419),film,"[reportáž, sportovní]","[dokumentární, distribuční]",...,None,,,,,,,,,24. 7. 1898


In [22]:
df.groupby(df['Premiéra bez upřesnění (datum)'].dt.weekday)["Film"].nunique().nlargest(50)

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
df.groupby("Festivalová premiéra (místo)")["Film"].nunique().nlargest(50)

In [ ]:
df.groupby("Copyright")["Slavnostní premiéra (místo)"].nunique().plot()

In [ ]:
top_kina = df.groupby("Slavnostní premiéra (místo)")["Film"].nunique().nlargest(10).dropna().index.to_list()

In [ ]:
top_kina = [x for x in top_kina if x != ""]

In [ ]:
df[(df["Slavnostní premiéra (místo)"].isin(top_kina)) & (df["Copyright"] > 1989)].groupby(["Copyright","Slavnostní premiéra (místo)"])["Slavnostní premiéra (datum)"].nunique().unstack().plot()

In [ ]:
kino_mask = df['Slavnostní premiéra (místo)'].str.contains('kino')
multiplex_mask = df['Slavnostní premiéra (místo)'].str.contains('multiplex')

In [ ]:
import numpy as np

In [ ]:
df['group'] = pd.Series(
    np.where(kino_mask, 'kino', None) +
    np.where(multiplex_mask, 'multiplex', None)
)

In [ ]:
df.groupby(["Copyright", df["Slavnostní premiéra (místo)"].str.contains("kino|multiplex")])["Slavnostní premiéra (datum)"].nunique().plot()

In [ ]:
df["Lokace"].to_list()